# Trabalho Prático

Imports

In [ ]:
!pip3 install graphviz

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn import tree
from sklearn.impute import KNNImputer

### Leitura do Arquivo CSV:

In [ ]:
data = pd.read_csv('heart2.csv')
data

### Tratamento de dados categóricos

In [ ]:

label_data = ['Sex', 'ExerciseAngina', 'Oldpeak', 'RestingECG', 'ST_Slope']

one_hot_data = ['ChestPainType']

lb = LabelEncoder() 


for col in label_data:
    data[col] = lb.fit_transform(data[col])
    
#Use pandas get_dummies for one-hot encoding
data = pd.get_dummies(data, columns=one_hot_data)

data

### Pre processamento

In [ ]:
imputer = KNNImputer(missing_values=-1)
imputer.fit_transform(data)


### Separar teste e treino

In [ ]:

x = data.drop(columns=["HeartDisease"])
y = data["HeartDisease"]

X_treino, X_teste, y_treino, y_teste = train_test_split(x, y, test_size = 0.2, random_state = 23)


## Árvore de decisão

### Criar modelo

In [ ]:
from sklearn.tree import DecisionTreeClassifier

modelo = DecisionTreeClassifier(criterion='entropy')
Y = modelo.fit(X_treino, y_treino)

#Teste do Modelo
prevision = modelo.predict(X_teste)


### Gerar matriz de confusão

In [ ]:
from yellowbrick.classifier import ConfusionMatrix
confusion_matrix(y_teste, prevision)
cm = ConfusionMatrix(modelo)
cm.fit(X_treino, y_treino)
cm.score(X_teste, y_teste)

cm

In [ ]:
print(classification_report(y_teste, prevision))
print(accuracy_score(y_teste, prevision))

### Plotando a árvore

In [ ]:
tree.plot_tree(Y)
plt.show()

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn import tree
from pprint import pprint

#Definição dos Hiperparâmetros
rf = RandomForestRegressor(random_state=23, n_jobs=-1)

parameters = {
    'n_estimators': [10, 50],
    'max_features': ['sqrt', 'log2'],
    'max_depth': [10, 50],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 3],
    'bootstrap': [True, False],
}

grid = GridSearchCV(rf, parameters, verbose=1, scoring='r2')
grid.fit(X_treino, y_treino)
print('Best Model: ' + str(grid.best_estimator_))


model = RandomForestClassifier(n_estimators=50, max_features='sqrt', random_state = 23, bootstrap = False, max_depth = 50, min_samples_leaf = 3, n_jobs = -1)
model.fit(X_treino, y_treino)
prevision2 = model.predict(X_teste)


In [ ]:
confusion_matrix(y_teste, prevision2)
cm2 = ConfusionMatrix(model)
cm2.fit(X_treino, y_treino)
cm2.score(X_teste, y_teste)

cm

In [ ]:
print(classification_report(y_teste, prevision2))
print(accuracy_score(y_teste, prevision2))

In [ ]:


for decision_tree in model.estimators_:
   tree.plot_tree(decision_tree)
   plt.show() 
  